## Dataset used :
- https://www.kaggle.com/datasets/asaniczka/video-game-sales-2024

## References
- https://www.kaggle.com/code/jruots/forecasting-video-game-sales
- 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load the dataset
data = pd.read_csv("vgchartz-2024.csv")

# Check for missing values
print(data.isnull().sum())

In [ ]:
#adjust output format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
print(pd.value_counts(data["console"]))

In [ ]:
print(data.shape)

In [ ]:
#check data points for any outliers between highly correleated features. In this case critic score and total sales

fig, ax = plt.subplots()
ax.scatter(x = data["critic_score"], y = data["total_sales"])
plt.ylabel("total_sales", fontsize=13)
plt.xlabel("critic_score", fontsize=13)
plt.show()



In [ ]:
# filter out the outliers. Filtering process is up to our discretion.
data = data.drop(data[(data["critic_score"] > 6) & (data["total_sales"] > 10)].index)
fig, ax = plt.subplots()
ax.scatter(x = data["critic_score"], y = data["total_sales"])
plt.ylabel("total_sales", fontsize=13)
plt.xlabel("critic_score", fontsize=13)
plt.show()

In [ ]:
#since total_sales is our target, we have to get rid of all rows with missing values for total_sales
data = data.dropna(subset=["total_sales"])

#fill in missing critic score values with the mean of critic scores for each genre
data["critic_score"]  = data.groupby("genre")['critic_score'].transform(

    lambda x: x.fillna(x.mean())
)

#Group publishers into "Top 10" and "Other" since there is too many unique publishers
top_publishers = data['publisher'].value_counts().head(10).index
data['publisher_grouped'] = data['publisher'].apply(

    lambda x: x if x in top_publishers else 'Other'

)

#convert the release date to date_time and extract the month and date from it and put it into separate columns
def parse_date(date):
    try:
        return pd.to_datetime(date, format="%d%b%y")
    except ValueError:
        return pd.to_datetime(date, format="%Y-%m-%d")

data["release_date"] = data["release_date"].apply(parse_date)
data["month"] = data["release_date"].dt.month
data["year"] = data["release_date"].dt.year

# Drop irrelevant columns
columns_to_drop = ['img', 'last_update', 'publisher', 'release_date', 'developer']

data = data.drop(columns=columns_to_drop)




In [ ]:
#One-hot encode categorical features and log-transform sales (still not sure if log transform is necessary)
data['Log_Total_Sales'] = np.log1p(data['total_sales'])

encoded_data = pd.get_dummies(data, columns=['console', 'genre', 'publisher_grouped'], drop_first=True)

X = encoded_data.drop(columns=['total_sales', 'Log_Total_Sales', 'title', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales'])

y = encoded_data['Log_Total_Sales']

# Handle missing Year and Month values
X['year'] = X['year'].fillna(X['year'].median())

X['month'] = X['month'].fillna(X['month'].mode()[0])

#split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:

models = {

    "Linear Regression": LinearRegression(),

    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),

    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

}

#linear model fit and training
models["Linear Regression"].fit(X_train,y_train)
y_pred = models["Linear Regression"].predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print("Linear Regression Performance:")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")


In [ ]:

#save the model
import joblib

# Save the model
joblib.dump(models["Linear Regression"], "linear_model.joblib")

# Load the model
loaded_model = joblib.load("linear_model.joblib")

# Use the loaded model for predictions
y_pred_loaded = loaded_model.predict(X_test)

print(y_pred_loaded)

In [ ]:
#still need to work on this part more 

# Example game data (replace with actual game details)
new_game = {
    "console": "PC",
    "genre": "Action",
    "Year": 2021,
    "Month": 11,
    "publisher_grouped": "Activision",
    "critic_score": 8.0
}
# Convert the input into a DataFrame
import pandas as pd

new_game_df = pd.DataFrame([new_game])

# One-hot encode categorical features
# Ensure one-hot encoding matches the training set
new_game_encoded = pd.get_dummies(new_game_df, columns=["console", "genre", "publisher_grouped"], drop_first=True)

# Align columns with the training data (X_train)
for col in X_train.columns:
    if col not in new_game_encoded:
        new_game_encoded[col] = 0  # Add missing columns with default values

# Reorder columns to match the training data
new_game_encoded = new_game_encoded[X_train.columns]
# Use the trained model to predict
predicted_log_sales = loaded_model.predict(new_game_encoded)

# Convert the log-transformed prediction back to the original scale
predicted_sales = np.expm1(predicted_log_sales)

print(f"Predicted Total Sales: {predicted_sales[0]:.2f} million units")


In [ ]:

#this module for hyperparameter tuning on the Random Forest and Gradient boost models
#not sure if necessary. need to play around with it more. 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Hyperparameter grids
param_grids = {
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [10, 20, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7],
        "subsample": [0.8, 1.0]
    }
}

# Results dictionary to store the best models and their performance
tuned_models = {}
results = []

for name, params in param_grids.items():
    if name == "Random Forest":
        model = RandomForestRegressor(random_state=42)
    elif name == "XGBoost":
        model = XGBRegressor(random_state=42)
    
    # Use RandomizedSearchCV for faster tuning
    search = RandomizedSearchCV(
        model,
        param_distributions=params,
        n_iter=20,  # Number of random combinations to try
        scoring="neg_mean_absolute_error",  # Use MAE for scoring
        cv=3,  # 3-fold cross-validation
        random_state=42,
        n_jobs=-1  # Use all available cores
    )
    
    # Fit the search
    search.fit(X_train, y_train)
    
    # Store the best model and its performance
    tuned_models[name] = search.best_estimator_
    best_params = search.best_params_
    best_score = -search.best_score_  # Convert negative MAE to positive
    
    results.append({"Model": name, "Best MAE": best_score, "Best Params": best_params})

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)



# Convert results to a DataFrame for display

results_df = pd.DataFrame(results)

print(results_df)


In [ ]:
# Train and evaluate the best models
final_results = []

for name, model in tuned_models.items():
    # Fit the model with the best parameters
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    final_results.append({"Model": name, "MAE": mae, "RMSE": rmse, "R²": r2})

# Display the final results
final_results_df = pd.DataFrame(final_results)

print(final_results_df)

In [ ]:
#scatterplot for the models

import matplotlib.pyplot as plt
import numpy as np

# Refined Scatter Plot
for name, model in models.items():
    # Get predictions
    y_pred = model.predict(X_test)
    
    # Scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.6, label="Predictions", color="blue")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color="red", linestyle="--", label="Perfect Prediction")
    plt.title(f"{name}: Predicted vs. Actual", fontsize=14)
    plt.xlabel("Actual Log Total Sales", fontsize=12)
    plt.ylabel("Predicted Log Total Sales", fontsize=12)
    plt.legend()
    plt.grid()
    
    # Add R² score as annotation
    r2 = r2_score(y_test, y_pred)
    plt.text(y_test.min(), y_test.max() - 0.5, f"R² = {r2:.2f}", fontsize=12, color="green")
    
    plt.show()


In [ ]:
# Refined Line Plot for the models. Need to work on this plot more.
sorted_indices = np.argsort(y_test.values)  # Sort by actual values for cleaner plotting

for name, model in models.items():
    # Get predictions
    y_pred = model.predict(X_test)
    
    # Line plot
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.values[sorted_indices], label="Actual", marker="o", linestyle="-", color="blue")
    plt.plot(y_pred[sorted_indices], label="Predicted", marker="x", linestyle="--", color="orange")
    plt.title(f"{name}: Actual vs. Predicted (Sorted)", fontsize=14)
    plt.xlabel("Sample Index (Sorted by Actual)", fontsize=12)
    plt.ylabel("Log Total Sales", fontsize=12)
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# Residual Plot for models
for name, model in models.items():
    # Get predictions
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred
    
    # Scatter residual plot
    plt.figure(figsize=(8, 6))
    plt.scatter(y_pred, residuals, alpha=0.6, color="purple")
    plt.axhline(0, color="red", linestyle="--", label="Zero Error")
    plt.title(f"{name}: Residuals Plot", fontsize=14)
    plt.xlabel("Predicted Log Total Sales", fontsize=12)
    plt.ylabel("Residuals (Actual - Predicted)", fontsize=12)
    plt.legend()
    plt.grid()
    plt.show()
    
    # Histogram of residuals
    plt.figure(figsize=(8, 6))
    plt.hist(residuals, bins=30, color="green", alpha=0.7, edgecolor="black")
    plt.title(f"{name}: Residuals Histogram", fontsize=14)
    plt.xlabel("Residuals", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    plt.grid()
    plt.show()


## Everything below is not necessary. It was my initial approach and is not being used. Still included for reference though. 

In [ ]:
data = data[(data['console'] == 'PC') | (data['console'] == 'PS4') 
            | (data['console'] == 'NS') 
            | (data['console'] == 'XBL')| (data['console'] == 'PSN')
            | (data['console'] == 'XOne')| (data['console'] == 'PS3')
            | (data['console'] == 'X360')
            | (data['console'] == 'Wii')]

print(pd.value_counts(data["console"]))

In [ ]:
data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data.head(16)

In [ ]:

# Drop rows with missing target values
data = data.dropna(subset=["total_sales","critic_score", "release_date", "developer"])
print(data.shape)

data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data.head(16)


In [ ]:

data = data[["console", "genre", "release_date", "publisher", "developer", "total_sales", "critic_score"]]

data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data.head(16)

In [ ]:
top_publishers = data["publisher"].value_counts().head(10).index
top_developers = data["developer"].value_counts().head(10).index


data["publisher"] = data["publisher"].apply(lambda x: x if x in top_publishers else "other_publishers")
data["developer"] = data["developer"].apply(lambda x: x if x in top_developers else "other_developers")



In [ ]:

# Drop rows with missing categorical data
data = data.dropna()

def parse_date(date):
    try:
        return pd.to_datetime(date, format="%d%b%y")
    except ValueError:
        return pd.to_datetime(date, format="%Y-%m-%d")

data["release_date"] = data["release_date"].apply(parse_date)
data["month"] = data["release_date"].dt.month
data["year"] = data["release_date"].dt.year

# Print first few rows
print(data.head())


In [ ]:
print(data.shape)
data = data.drop(columns=["release_date"])
# Encode categorical variables
data = pd.get_dummies(data, columns=["console", "genre", "developer", "publisher"], drop_first=True)

print(data.shape)
# Print encoded data
print(data.head())


In [ ]:
# Define features and target
X = data.drop(columns=["total_sales"])
Y = data["total_sales"]
print(X.shape)
print(Y.shape)

In [ ]:

# Split into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"XTraining set size: {X_train.shape}")
print(f"XTest set size: {X_test.shape}")
print(f"YTraining set size: {Y_train.shape}")
print(f"YTest set size: {Y_test.shape}")


In [ ]:
# publisher_mean_sales =  X_train.join(Y_train).groupby("publisher")["total_sales"].mean()

# X_train["publisher_encoded"] = X_train["publisher"].map(publisher_mean_sales)
# X_test["publisher_encoded"] = X_test["publisher"].map(publisher_mean_sales)

# developer_mean_sales =  X_train.join(Y_train).groupby("developer")["total_sales"].mean()

# X_train["developer_encoded"] = X_train["developer"].map(developer_mean_sales)
# X_test["developer_encoded"] = X_test["developer"].map(developer_mean_sales)

# # Handle missing encodings in test set
# overall_mean = Y_train.mean()
# X_test["publisher_encoded"] = X_test["publisher_encoded"].fillna(overall_mean)

# overall_mean_developer = Y_train.mean()
# X_test["developer_encoded"] = X_test["developer_encoded"].fillna(overall_mean_developer)

# X_train = X_train.drop(columns=["publisher"])
# X_test = X_test.drop(columns=["publisher"])


# X_train = X_train.drop(columns=["developer"])
# X_test = X_test.drop(columns=["developer"])




In [ ]:
from scipy import stats
from scipy.stats import norm
sns.displot(data["total_sales"], fit=norm);

(mu, sigma) = norm.fit(data["total_sales"])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
print(f"Test set size: {X_test.shape}")

plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('total_Sales distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(data['total_sales'], plot=plt)
plt.show()

In [ ]:
# Train the model
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

# Predict on test data
linear_predictions = linear_model.predict(X_test)

# Evaluate the model
mae_linear = mean_absolute_error(Y_test, linear_predictions)
rmse_linear = mean_squared_error(Y_test, linear_predictions, squared=False)
r2_linear = r2_score(Y_test, linear_predictions)

print("Linear Regression Performance:")
print(f"MAE: {mae_linear}")
print(f"RMSE: {rmse_linear}")
print(f"R² Score: {r2_linear}")


In [ ]:
# Train the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)


In [ ]:
importances = rf_model.feature_importances_
feature_names = X_train.columns

feature_importance_df =  pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)
print(feature_importance_df.head(20))

# Plot feature importance
plt.figure(figsize=(10, 10))
plt.barh(feature_importance_df["Feature"].head(20), feature_importance_df["Importance"].head(20))
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Top 10 Important Features (Random Forest)")
plt.gca().invert_yaxis()  # Flip the y-axis for better readability
plt.show()

In [ ]:
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# Train the model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, Y_train)

# Get feature importance
importances = xgb_model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

# Print top 10 features
print(feature_importance_df.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df["Feature"].head(10), feature_importance_df["Importance"].head(10))
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Top 10 Important Features (XGBoost)")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
from sklearn.inspection import permutation_importance

# Compute permutation importance
perm_importance = permutation_importance(rf_model, X_test, Y_test, n_repeats=10, random_state=42)

# Create a DataFrame
perm_importance_df = pd.DataFrame({
    "Feature": X_test.columns,
    "Importance": perm_importance.importances_mean
}).sort_values(by="Importance", ascending=False)

# Print top 10 features
print(perm_importance_df.head(10))

# Plot permutation importance
plt.figure(figsize=(10, 6))
plt.barh(perm_importance_df["Feature"].head(10), perm_importance_df["Importance"].head(10))
plt.xlabel("Permutation Importance")
plt.ylabel("Feature")
plt.title("Top 10 Features (Permutation Importance)")
plt.gca().invert_yaxis()
plt.show()


In [ ]:

# Predict on test data
rf_predictions = rf_model.predict(X_test)

# Evaluate the model
mae_rf = mean_absolute_error(Y_test, rf_predictions)
rmse_rf = mean_squared_error(Y_test, rf_predictions, squared=False)
r2_rf = r2_score(Y_test, rf_predictions)



In [ ]:
print("Random Forest Performance:")
print(f"MAE: {mae_rf}")
print(f"RMSE: {rmse_rf}")
print(f"R² Score: {r2_rf}")


In [ ]:
# Train the model
xgb_model = XGBRegressor(n_estimators=200, learning_rate=0.2, max_depth=10, random_state=42)
xgb_model.fit(X_train, Y_train)

# Predict on test data
xgb_predictions = xgb_model.predict(X_test)

# Evaluate the model
mae_xgb = mean_absolute_error(Y_test, xgb_predictions)
rmse_xgb = mean_squared_error(Y_test, xgb_predictions, squared=False)
r2_xgb = r2_score(Y_test, xgb_predictions)

print("XGBoost Performance:")
print(f"MAE: {mae_xgb}")
print(f"RMSE: {rmse_xgb}")
print(f"R² Score: {r2_xgb}")


In [ ]:
# Print comparison of metrics
results = {
    "Model": ["Linear Regression", "Random Forest", "XGBoost"],
    "MAE": [mae_linear, mae_rf, mae_xgb],
    "RMSE": [rmse_linear, rmse_rf, rmse_xgb],
    "R² Score": [r2_linear, r2_rf, r2_xgb]
}

results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
# Scatter plot for actual vs. predicted sales (e.g., for Random Forest)
plt.scatter(Y_test, rf_predictions, alpha=0.5)
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.title("Actual vs. Predicted Sales (Random Forest)")
plt.xlim(0,6)
plt.ylim(0,6)
plt.show()


In [ ]:
# Feature importance for Random Forest
feature_importances = rf_model.feature_importances_
features = X.columns

# Plot feature importance
plt.barh(features, feature_importances)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance (Random Forest)")
plt.show()
